In [6]:
# Variable for currently active object
myobj = bpy.context.object
# Alternatively, if you know that the object is called 'Cube'
# you can reach it by
# myobj = bpy.data.objects['Cube']

# Clear all previous animation data
myobj.animation_data_clear()

# set first and last frame index
total_time = 2*pi # Animation should be 2*pi seconds long
fps = 24 # Frames per second (fps)
bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = round(total_time*fps)+1

# loop of frames and insert keyframes at every frame
nlast = bpy.context.scene.frame_end
for n in range(nlast):
    t = total_time*n/nlast

    # Set frame like this
    bpy.context.scene.frame_set(n)
    
    # 𝐫(𝑡)=[5cos(𝑡),5sin(𝑡),sin(5𝑡)]
    
    # Set current location like this
    myobj.location.x = 5*cos(t)
    myobj.location.y = 5*sin(t)
    myobj.location.z = 5*sin(t)
    # myobj.location.x = ...
    # myobj.location.y = ...
    # myobj.location.z = ...

    # Insert new keyframe for "location" like this
    myobj.keyframe_insert(data_path="location")

## Task 3: Rigid body dynamics in Blender by Python scripting 

In this exercise you will visualize the unstable relative equilibrium of a free rigid in Blender by using Python scripting (see [this presentation](https://slides.com/kmodin/mmg640-velocity-16-18?token=eol1jbFe)). 
Create an ellipsoid with half axes $(0.5, 2, 4)$. 
The easiest way is to first create a sphere and then flatten or extrude it differently in the $x$, $y$, and $z$ directions. 
The task is to create a Python script that solves the Euler equations for a rigid body in quaternion formulation using the Lie-Euler method (as explained in the presentation), and from this solution generate rotation keyframes in Blender for the object in your scene. The physically correct way to do this is to first calculate the moments of intertia for your object. However, as this is a little complicated to do in Blender, I give you the moments of inertia tensor: it is diagonal with diagonal elements $(0.5, 2, 4)$. 
This corresponds to your ellipsoid object. Here is the data for the full problem:

- The initial data is $q_0 = (1,0,0,0)$ and $\omega=(10^{-2},1,0)$.
- The total simulation time interval is $[0, 25]$.
- The stepsize is $h = 0.01$.
- Make your simulation so that each frame in Blender corresponds to 10 time steps with the Lie-Euler method.

Here is some additional information that will help you get on track:
- Make sure everything from the `math` and `mathutils` modules are loaded. They support quaternions through the class `Quaternion`. For example, to create and multiply two quaternions you do the following
```python
q1 = Quaternion((1.0, 2.0, 3.0, 4.0))
q2 = Quaternion((0.0, 0.5, 1.5, -0.5))
q3 = q1@q2
```
- To select that Blender should use quaternions for rotations for the object in the variable `myobj`, do
`myobj.rotation_mode = 'QUATERNION'`. After that, to get or set the quaternion representing the orientation of the currently selected object, do 
`q = myobj.rotation_quaternion`, or, to set it, do
`myobj.rotation_quaternion = q`.
- For the `omega` variable (3 vector) it is convenient to use the `Vector` class (also available through `mathutils`). This class also supports the dot and cross products. For example, to create two vectors and take their cross product, do 
```python
omega1 = Vector((1.0, -3.0, 0.0))
omega2 = Vector((0.0, 2.0, -2.0))
omega3 = omega1.cross(omega2)
```
- Likewise, matrices can be created from the `Matrix` class. To construct the diagonal inertia tensor, do 
`I = Matrix(((0.5,0,0),(0,2,0),(0,0,4)))`. Its inverse can be computed with
`I.inverted()`. Multiplication with a `omega` (object from class `Vector`) is obtained as
`I@omega`.
- The `qexp` function, returning the quaternion corresponding to the matrix exponential ([see slides](https://slides.com/kmodin/mmg640-velocity-16-18?token=eol1jbFe)) is obtained by sending in a vector of length 3 (corresponding to `omega`) to the constructor of the `Quaternion` class. For example, if `omeg` is a `Vector` object, then `qexp(omega)` is computed by
`q = Quaternion(omega)`.

In [5]:
import bpy
from math import *
from mathutils import *
I = Matrix(((0.5,0,0), (0,2,0), (0,0,4)))

h = 0.01
omega = Vector((1/100,1,0))
q = Quaternion((1,0,0,0))

# Variable for currently active object
myobj = bpy.context.object

# Clear all previous animation data
myobj.animation_data_clear()

myobj.rotation_mode = 'QUATERNION'

# Reset the location
myobj.location.x = 0
myobj.location.y = 0
myobj.location.z = 0

# set first and last frame index
total_time = 25 # 25 seconds
fps = 24 # Frames per second (fps)
num_frames = total_time*fps # 600

bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = num_frames

# loop of frames and insert keyframes at every frame
nlast = bpy.context.scene.frame_end

time_step_counter = 0
n_timesteps = 10 * num_frames 

for frame_number in range(1, num_frames+1):
#     print(q)
    # Start every frame with setting the frames
    bpy.context.scene.frame_set(frame_number)
    
    # Set the rotation
    print(myobj.rotation_quaternion)
    myobj.rotation_quaternion = q
    
    # Set keyframe
    a = myobj.keyframe_insert(data_path="rotation_quaternion") # change to rotation
#     print(a)
    # iterate dynamics 10 times
    for _ in range(10):
    
        omega_dot = -1*I.inverted() @ (omega.cross(I @ omega))
        omega = omega - h*omega_dot
        qexp = Quaternion(h*omega)
        q = q @ qexp


<Quaternion (w=0.9007, x=-0.0024, y=-0.4344, z=0.0042)>
<Quaternion (w=1.0000, x=0.0000, y=0.0000, z=0.0000)>
<Quaternion (w=0.9988, x=0.0005, y=0.0500, z=0.0000)>
<Quaternion (w=0.9950, x=0.0010, y=0.0998, z=0.0000)>
<Quaternion (w=0.9888, x=0.0015, y=0.1494, z=0.0001)>
<Quaternion (w=0.9801, x=0.0021, y=0.1987, z=0.0001)>
<Quaternion (w=0.9689, x=0.0027, y=0.2474, z=0.0002)>
<Quaternion (w=0.9553, x=0.0033, y=0.2955, z=0.0003)>
<Quaternion (w=0.9394, x=0.0039, y=0.3429, z=0.0004)>
<Quaternion (w=0.9211, x=0.0046, y=0.3894, z=0.0005)>
<Quaternion (w=0.9004, x=0.0054, y=0.4350, z=0.0006)>
<Quaternion (w=0.8776, x=0.0063, y=0.4794, z=0.0007)>
<Quaternion (w=0.8525, x=0.0072, y=0.5227, z=0.0008)>
<Quaternion (w=0.8253, x=0.0082, y=0.5646, z=0.0008)>
<Quaternion (w=0.7961, x=0.0094, y=0.6052, z=0.0008)>
<Quaternion (w=0.7648, x=0.0106, y=0.6442, z=0.0008)>
<Quaternion (w=0.7316, x=0.0120, y=0.6816, z=0.0006)>
<Quaternion (w=0.6966, x=0.0135, y=0.7173, z=0.0005)>
<Quaternion (w=0.6599, x=0

<Quaternion (w=0.0628, x=0.2687, y=0.0689, z=0.9587)>
<Quaternion (w=0.0667, x=0.3190, y=0.0818, z=0.9419)>
<Quaternion (w=0.0703, x=0.3682, y=0.0966, z=0.9220)>
<Quaternion (w=0.0734, x=0.4160, y=0.1137, z=0.8993)>
<Quaternion (w=0.0758, x=0.4621, y=0.1332, z=0.8735)>
<Quaternion (w=0.0774, x=0.5063, y=0.1554, z=0.8447)>
<Quaternion (w=0.0777, x=0.5483, y=0.1804, z=0.8129)>
<Quaternion (w=0.0765, x=0.5875, y=0.2086, z=0.7781)>
<Quaternion (w=0.0732, x=0.6237, y=0.2400, z=0.7403)>
<Quaternion (w=0.0675, x=0.6563, y=0.2747, z=0.6995)>
<Quaternion (w=0.0589, x=0.6847, y=0.3127, z=0.6557)>
<Quaternion (w=0.0468, x=0.7082, y=0.3539, z=0.6091)>
<Quaternion (w=0.0306, x=0.7262, y=0.3978, z=0.5598)>
<Quaternion (w=0.0099, x=0.7379, y=0.4439, z=0.5083)>
<Quaternion (w=-0.0158, x=0.7426, y=0.4913, z=0.4549)>
<Quaternion (w=-0.0467, x=0.7397, y=0.5389, z=0.4003)>
<Quaternion (w=-0.0828, x=0.7290, y=0.5852, z=0.3453)>
<Quaternion (w=-0.1241, x=0.7104, y=0.6288, z=0.2909)>
<Quaternion (w=-0.1698, 